In [2]:
import os
from dotenv import load_dotenv

load_dotenv()

SANDBOX_TOKEN = os.getenv("SANDBOX_TOKEN")

if not SANDBOX_TOKEN:
    raise RuntimeError("SANDBOX_TOKEN not found in .env")

In [6]:
import logging
import csv
from t_tech.invest import Client, InstrumentStatus

TOKEN = SANDBOX_TOKEN
logging.basicConfig(level=logging.INFO)

OUT_FILE = "shares_rub_common.csv"

def main():
    rows = []

    with Client(TOKEN) as client:
        resp = client.instruments.shares(
            instrument_status=InstrumentStatus.INSTRUMENT_STATUS_BASE,
            instrument_exchange=InstrumentStatus.INSTRUMENT_STATUS_UNSPECIFIED,
        )

        for s in resp.instruments:
            if s.currency == "rub" and "привил" not in s.name.lower():
                rows.append({
                    "figi": s.figi,
                    "ticker": s.ticker,
                    "name": s.name,
                    "issue_size": s.issue_size,
                    "uid": s.uid,
                    "assetUid": s.asset_uid,
                })

    with open(OUT_FILE, "w", newline="", encoding="utf-8-sig") as f:
        writer = csv.DictWriter(
            f,
            fieldnames=["figi", "ticker", "name", "issue_size", "currency", "uid", "assetUid"],
        )
        writer.writeheader()
        writer.writerows(rows)

    print(f"Saved {len(rows)} rows to {OUT_FILE}")

if __name__ == "__main__":
    main()


Saved 151 rows to shares_rub_common.csv


In [ ]:
import csv
import logging
import os
import re
from datetime import timedelta
from pathlib import Path

from t_tech.invest import CandleInterval, Client
from t_tech.invest.caching.market_data_cache.cache import MarketDataCache
from t_tech.invest.caching.market_data_cache.cache_settings import MarketDataCacheSettings
from t_tech.invest.utils import now

TOKEN = SANDBOX_TOKEN  
logging.basicConfig(format="%(levelname)s: %(message)s", level=logging.INFO)

INPUT_CSV = "shares_rub_common.csv"  
OUT_DIR = Path("stocks")
CACHE_DIR = Path("market_data_cache")

DAYS = 365
INTERVAL = CandleInterval.CANDLE_INTERVAL_DAY


def safe_filename(s: str) -> str:
    s = s.strip()
    s = re.sub(r'[<>:"/\\|?*\x00-\x1F]', "_", s) 
    s = re.sub(r"\s+", " ", s)
    return s[:150] if len(s) > 150 else s


def q_to_float(q) -> float | None:
    if q is None:
        return None
    try:
        return float(q.units) + float(q.nano) / 1_000_000_000
    except Exception:
        return None


def read_figis_from_csv(path: str):
    items = []
    with open(path, "r", encoding="utf-8-sig", newline="") as f:
        reader = csv.DictReader(f)
        for row in reader:
            figi = (row.get("figi") or "").strip()
            ticker = (row.get("ticker") or "").strip()
            name = (row.get("name") or "").strip()
            if figi:
                items.append((figi, ticker, name))
    return items


def save_candles_for_figi(market_data_cache: MarketDataCache, figi: str, out_path: Path):
    candles = list(
        market_data_cache.get_all_candles(
            figi=figi,
            from_=now() - timedelta(days=DAYS),
            interval=INTERVAL,
        )
    )

    out_path.parent.mkdir(parents=True, exist_ok=True)
    with open(out_path, "w", encoding="utf-8-sig", newline="") as f:
        w = csv.writer(f)
        w.writerow(["time", "open", "high", "low", "close", "volume", "is_complete"])
        for c in candles:
            w.writerow([
                c.time.isoformat() if c.time else None,
                q_to_float(getattr(c, "open", None)),
                q_to_float(getattr(c, "high", None)),
                q_to_float(getattr(c, "low", None)),
                q_to_float(getattr(c, "close", None)),
                getattr(c, "volume", None),
                getattr(c, "is_complete", None),
            ])

    return len(candles)


def main():
    OUT_DIR.mkdir(parents=True, exist_ok=True)

    items = read_figis_from_csv(INPUT_CSV)
    if not items:
        raise RuntimeError(f"No FIGIs found in {INPUT_CSV}")

    with Client(TOKEN) as client:
        settings = MarketDataCacheSettings(base_cache_dir=CACHE_DIR)
        market_data_cache = MarketDataCache(settings=settings, services=client)

        ok = 0
        for figi, ticker, name in items:
            label = ticker or name or figi
            fname = safe_filename(f"{ticker or 'TICKER'}_{figi}.csv")
            out_path = OUT_DIR / fname

            try:
                n = save_candles_for_figi(market_data_cache, figi, out_path)
                logging.info("Saved %s candles for %s (%s) -> %s", n, label, figi, out_path)
                ok += 1
            except Exception as e:
                logging.exception("Failed for %s (%s): %s", label, figi, e)

    logging.info("Done. Successfully saved: %d / %d", ok, len(items))


if __name__ == "__main__":
    main()


In [30]:
import logging
from t_tech.invest import Client, GetAssetFundamentalsRequest

TOKEN = SANDBOX_TOKEN
logging.basicConfig(level=logging.INFO)

ASSET_UID = "f712e646-7c5d-4266-aabd-a18c9c65a7e2"

def main():
    with Client(TOKEN) as client:
        resp = client.instruments.get_asset_fundamentals(GetAssetFundamentalsRequest(
            assets=[ASSET_UID])
        )

        print(resp)

if __name__ == "__main__":
    main()


INFO:t_tech.invest.logging:945ddf6f52acf2cbe02be37fb70fde76 GetAssetFundamentals


GetAssetFundamentalsResponse(fundamentals=[StatisticResponse(asset_uid='f712e646-7c5d-4266-aabd-a18c9c65a7e2', currency='RUB', market_capitalization=365947536120.0, high_price_last_52_weeks=43400.0, low_price_last_52_weeks=24260.0, average_daily_volume_last_10_days=1545.22, average_daily_volume_last_4_weeks=1273.68, beta=1.0, free_float=0.1, forward_annual_dividend_yield=0.0, shares_outstanding=11529538.0, revenue_ttm=108271800000.0, ebitda_ttm=0.0, net_income_ttm=6889243000.0, eps_ttm=597.53, diluted_eps_ttm=0.0, free_cash_flow_ttm=3964329000.0, five_year_annual_revenue_growth_rate=0.0, three_year_annual_revenue_growth_rate=0.0, pe_ratio_ttm=53.12, price_to_sales_ttm=3.38, price_to_book_ttm=1.21, price_to_free_cash_flow_ttm=92.31, total_enterprise_value_mrq=397498692120.0, ev_to_ebitda_mrq=0.0, net_margin_mrq=6.36, net_interest_margin_mrq=0.0, roe=2.26, roa=1.46, roic=0.0, total_debt_mrq=63814414000.0, total_debt_to_equity_mrq=21.14, total_debt_to_ebitda_mrq=0.0, free_cash_flow_to_pri

In [ ]:
import csv
import logging
from datetime import date, datetime
from pathlib import Path

from t_tech.invest import Client

TOKEN = SANDBOX_TOKEN
logging.basicConfig(format="%(levelname)s: %(message)s", level=logging.INFO)

INPUT_CSV = "shares_rub_common.csv"
OUT_CSV = "asset_fundamentals_all.csv"

BATCH_SIZE = 50 


def iso(v):
    if isinstance(v, (datetime, date)):
        return v.isoformat()
    return v


def read_asset_uids(path: str):
    uids = []
    with open(path, "r", encoding="utf-8-sig", newline="") as f:
        r = csv.DictReader(f)

        possible_cols = ["assetUid", "asset_uid", "uid"]
        cols = r.fieldnames or []
        col = next((c for c in possible_cols if c in cols), None)
        if not col:
            raise RuntimeError(
                f"Не нашёл колонку assetUid/asset_uid/uid в CSV. Есть колонки: {cols}"
            )

        for row in r:
            uid = (row.get(col) or "").strip()
            if uid:
                uids.append(uid)

    seen = set()
    out = []
    for u in uids:
        if u not in seen:
            seen.add(u)
            out.append(u)
    return out


def obj_to_row(obj):
    d = {}
    for k, v in vars(obj).items():
        d[k] = iso(v)
    return d


def chunked(xs, n):
    for i in range(0, len(xs), n):
        yield xs[i:i + n]


def main():
    asset_uids = read_asset_uids(INPUT_CSV)
    logging.info("Loaded asset UIDs: %d", len(asset_uids))

    all_rows = []
    with Client(TOKEN) as client:
        for batch in chunked(asset_uids, BATCH_SIZE):
            resp = client.instruments.get_asset_fundamentals(GetAssetFundamentalsRequest(assets=batch))

            for f in resp.fundamentals:
                all_rows.append(obj_to_row(f))

            logging.info("Fetched fundamentals: +%d (total %d)", len(resp.fundamentals), len(all_rows))

    if not all_rows:
        raise RuntimeError("Фундаментальные показатели не вернулись ни для одного UID.")

    fieldnames = sorted({k for row in all_rows for k in row.keys()})

    with open(OUT_CSV, "w", encoding="utf-8-sig", newline="") as f:
        w = csv.DictWriter(f, fieldnames=fieldnames)
        w.writeheader()
        w.writerows(all_rows)

    logging.info("Saved %d rows to %s", len(all_rows), OUT_CSV)


if __name__ == "__main__":
    main()


INFO:root:Loaded asset UIDs: 151
INFO:t_tech.invest.logging:e8482108f2c1f7d7fbfef8ca5c5dd937 GetAssetFundamentals
INFO:root:Fetched fundamentals: +50 (total 50)
INFO:t_tech.invest.logging:d14d21532828cbd6e4d4f5c2cb7e4a98 GetAssetFundamentals
INFO:root:Fetched fundamentals: +49 (total 99)
INFO:t_tech.invest.logging:60df7b6e4089661ab30420da426aa328 GetAssetFundamentals
INFO:root:Fetched fundamentals: +48 (total 147)
INFO:t_tech.invest.logging:aac1095ceca17d2fc8df87cd805ab143 GetAssetFundamentals
INFO:root:Fetched fundamentals: +1 (total 148)
INFO:root:Saved 148 rows to asset_fundamentals_all.csv


In [ ]:
import pandas as pd

SHARES_FILE = "shares_rub_common.csv"
FUND_FILE = "asset_fundamentals_all.csv"
OUT_FILE = "stocks_summary.csv"

def main():
    shares = pd.read_csv(SHARES_FILE, encoding="utf-8-sig")
    funds = pd.read_csv(FUND_FILE, encoding="utf-8-sig")

    shares = shares.rename(columns={"assetUid": "asset_uid"})

    df = shares.merge(
        funds,
        how="left",
        on="asset_uid",
    )

    preferred_cols = [
        "figi",
        "ticker",
        "name",
        "asset_uid",
        "issue_size",
        "shares_outstanding",
        "free_float",
        "market_capitalization",
        "pe_ratio_ttm",
        "price_to_book_ttm",
        "price_to_sales_ttm",
        "roe",
        "roa",
        "net_margin_mrq",
        "revenue_ttm",
        "net_income_ttm",
        "total_debt_mrq",
        "total_debt_to_equity_mrq",
        "average_daily_volume_last_10_days",
        "average_daily_volume_last_4_weeks",
        "high_price_last_52_weeks",
        "low_price_last_52_weeks",
        "currency",
    ]

    first_cols = [c for c in preferred_cols if c in df.columns]

    rest_cols = [c for c in df.columns if c not in first_cols]

    df = df[first_cols + rest_cols]

    df.to_csv(OUT_FILE, index=False, encoding="utf-8-sig")

    print("Saved summary table:", OUT_FILE)
    print("Rows:", len(df))
    print("Columns:", len(df.columns))

    missing = [c for c in preferred_cols if c not in df.columns]
    if missing:
        print("Missing columns (ok):", missing)

if __name__ == "__main__":
    main()


Saved summary table: stocks_summary.csv
Rows: 151
Columns: 62
Missing columns (ok): ['currency']


In [40]:
import pandas as pd

pd.set_option("display.max_columns", None)
pd.set_option("display.width", 0)

df = pd.read_csv("stocks_summary.csv", encoding="utf-8-sig")


In [44]:
df.columns

Index(['figi', 'ticker', 'name', 'asset_uid', 'issue_size',
       'shares_outstanding', 'free_float', 'market_capitalization',
       'pe_ratio_ttm', 'price_to_book_ttm', 'price_to_sales_ttm', 'roe', 'roa',
       'net_margin_mrq', 'revenue_ttm', 'net_income_ttm', 'total_debt_mrq',
       'total_debt_to_equity_mrq', 'average_daily_volume_last_10_days',
       'average_daily_volume_last_4_weeks', 'high_price_last_52_weeks',
       'low_price_last_52_weeks', 'currency_x', 'uid',
       'adr_to_common_share_ratio', 'beta', 'buy_back_ttm', 'currency_y',
       'current_ratio_mrq', 'diluted_eps_ttm', 'dividend_payout_ratio_fy',
       'dividend_rate_ttm', 'dividend_yield_daily_ttm', 'dividends_per_share',
       'domicile_indicator_code', 'ebitda_change_five_years', 'ebitda_ttm',
       'eps_change_five_years', 'eps_ttm', 'ev_to_ebitda_mrq', 'ev_to_sales',
       'ex_dividend_date', 'fiscal_period_end_date',
       'fiscal_period_start_date', 'five_year_annual_dividend_growth_rate',
      

In [41]:
cols = [
    "ticker",
    "name",
    "market_capitalization",
    "pe_ratio_ttm",
    "price_to_book_ttm",
    "roe",
    "roa",
    "net_margin_mrq",
    "revenue_ttm",
    "total_debt_to_equity_mrq",
]

df[cols].head(5).style.format({
    "market_capitalization": "{:,.0f}",
    "revenue_ttm": "{:,.0f}",
    "pe_ratio_ttm": "{:.2f}",
    "roe": "{:.2f}",
    "roa": "{:.2f}",
    "net_margin_mrq": "{:.2f}",
    "total_debt_to_equity_mrq": "{:.2f}",
})

,ticker,name,market_capitalization,pe_ratio_ttm,price_to_book_ttm,roe,roa,net_margin_mrq,revenue_ttm,total_debt_to_equity_mrq
0,VSMO,ВСМПО-АВИСМА,"365,947,536,120",53.12,1.210000,2.26,1.46,6.36,"108,271,800,000",21.14
1,UNAC,Объединенная авиастроительная корпорация,"490,047,673,786",0.00,4.930000,-15.40,-0.54,-2.06,"599,897,000,000",833.67
2,CNRU,Циан,"44,660,531,750",0.00,4.060000,28.68,21.11,0.00,0,0.00
3,VKCO,ВК,"181,066,366,089",0.00,1.260000,-82.56,-23.74,-55.35,"149,952,000,000",72.06
4,MGNT,Магнит,"331,517,637,815",12.26,1.800000,15.68,1.75,0.83,"3,256,598,788,000",274.84
